<a href="https://colab.research.google.com/github/dhrruvchotai/Current_Learnings/blob/main/Pytorch/fashion_mnist_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import kagglehub
path = kagglehub.dataset_download("zalando-research/fashionmnist")

Using Colab cache for faster access to the 'fashionmnist' dataset.


In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import matplotlib.pyplot as plt

In [35]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
device

'cuda'

In [36]:
df = pd.read_csv("/kaggle/input/fashionmnist/fashion-mnist_train.csv")

In [37]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
df.shape

(60000, 785)

In [39]:
X = df.drop(columns=["label"])
y = df["label"]

In [40]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [41]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [42]:
X_train

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
48572,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
38696,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
13611,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
35213,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
31766,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.003922,0.0,...,0.000000,0.000000,0.003922,0.000000,0.043137,0.145098,0.023529,0.000000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54343,0.0,0.0,0.0,0.0,0.0,0.011765,0.027451,0.000000,0.000000,0.0,...,0.682353,0.705882,0.768627,0.921569,0.000000,0.000000,1.000000,0.898039,0.65098,0.0
38158,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
860,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.003922,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0
15795,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.007843,0.000000,0.0,...,0.368627,0.290196,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0


In [43]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features = torch.tensor(features.values,dtype=torch.float32)
    self.labels = torch.tensor(labels.values,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index],self.labels[index]

In [44]:
train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)

In [45]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [46]:
class Model(nn.Module):

  def __init__(self,num_features):
    super().__init__()
    self.neural_net = nn.Sequential(
        nn.Linear(num_features,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(64,10)
        #will not add softmax here because there is softmax function in built in cross entropy loss in pytorch
    )

  def forward(self,features):
    return self.neural_net(features)


In [47]:
# !pip install torchinfo

In [48]:
from torchinfo import summary

In [49]:
summary(model)

Layer (type:depth-idx)                   Param #
Model                                    --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       100,480
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       8,256
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0

In [50]:
num_features = X_train.shape[1]
model = Model(num_features)
model = model.to(device)

In [51]:
epochs = 100
learning_rate = 0.1

In [52]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),learning_rate,weight_decay=1e-4)

In [53]:
num_train_batches = len(train_loader)
num_train_batches

1500

In [54]:
#training loop
for epoch in range(epochs):
  total_epoch_loss = 0
  for batch_features,batch_labels in train_loader:
    batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
    #forward pass
    y_pred = model(batch_features)
    #cal loss
    loss = loss_function(y_pred,batch_labels)
    #clear gradients
    optimizer.zero_grad()
    #backprop
    loss.backward()
    #update params
    optimizer.step()
    total_epoch_loss += loss.item()

  print(f"Epoch {epoch+1} Loss : {total_epoch_loss/num_train_batches}")

Epoch 1 Loss : 0.6280991504093011
Epoch 2 Loss : 0.49664195294181507
Epoch 3 Loss : 0.4595259101887544
Epoch 4 Loss : 0.4369149357328812
Epoch 5 Loss : 0.4180646397123734
Epoch 6 Loss : 0.4010362710058689
Epoch 7 Loss : 0.39394312631338835
Epoch 8 Loss : 0.38328088905413943
Epoch 9 Loss : 0.3791791297495365
Epoch 10 Loss : 0.36968427170316376
Epoch 11 Loss : 0.36752843808134394
Epoch 12 Loss : 0.3575837296197812
Epoch 13 Loss : 0.35670711563030877
Epoch 14 Loss : 0.34937676140169305
Epoch 15 Loss : 0.34577189821501575
Epoch 16 Loss : 0.3423099287847678
Epoch 17 Loss : 0.33805158994098505
Epoch 18 Loss : 0.3347852082600196
Epoch 19 Loss : 0.3317051184649269
Epoch 20 Loss : 0.3282798137863477
Epoch 21 Loss : 0.3250390075022976
Epoch 22 Loss : 0.31913406605025135
Epoch 23 Loss : 0.3215528582533201
Epoch 24 Loss : 0.32056358314553896
Epoch 25 Loss : 0.3160847417662541
Epoch 26 Loss : 0.314533903097113
Epoch 27 Loss : 0.31115174741297963
Epoch 28 Loss : 0.31272959483414886
Epoch 29 Loss : 0

In [55]:
#To Check how torch.max function works
# torch.manual_seed(42)
# outputs = torch.rand(32,10)
# print(outputs[0])

In [56]:
# _, output = torch.max(outputs,dim=1)
# print(f"{_} and {output}")

In [57]:
model.eval()

Model(
  (neural_net): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [58]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
    outputs = model(batch_features)
    _, predicted = torch.max(outputs,dim=1)
    total = total + batch_features.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()
print(correct/total)

0.8949166666666667


In [59]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in train_loader:
    batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
    outputs = model(batch_features)
    _, predicted = torch.max(outputs,dim=1)
    total = total + batch_features.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()
print(correct/total)

0.9466875
